# Advanced Usage

Here we will introduce some advanced usage of QSPARSE by topics.

## Vector Quantization

## Structured Pruning

## Saturated Quantization

## Bias Quantization

## Inspecting Parameters of a Pruned/Quantized Model

## Window Size for Feature Pruning (and Quantization)

## Resuming from Checkpoint 

## Better Logging

## Extending Methods of Quantization And Pruning